In [1]:
from pathlib import Path

from cleantext import clean
import dataset
import requests

In [2]:
auth = tuple(Path('secrets.txt').read_text().split()[1:])

In [3]:
all_incidents = []
all_src = []

for p in Path('data').glob('*'):
    print(p)
    try:
        db = dataset.connect('sqlite:///'  + str(p))
        tab_incidents = db['incidents'].all()
        tab_src = db['sources'].all()
        all_incidents += tab_incidents
        all_src += tab_src
    except:
        print('error')

data/data_0.db
data/data_1.db
data/data_2.db


In [4]:
db = dataset.connect('sqlite:///rechtegewalt.db')

tab_incidents = db['incidents']
tab_incidents.insert_many(all_incidents)

tab_src = db['sources']
tab_src.insert_many(all_src)

tab_incidents.create_index(['identifier'])
tab_src.create_index(['identifier'])

In [5]:
# clean location text because there were still some errors
for x in tab_incidents.all():
    x['subdivisions'] = clean(x['subdivisions'], lang='de', lower=False)
    tab_incidents.update(x, ['id'])

In [6]:
subs = tab_incidents.distinct('subdivisions')

In [7]:
subs = list(map(lambda x: x['subdivisions'], subs))

In [8]:
r = requests.post(f'https://geocode.app.vis.one/', auth=auth, json={'locations': [{'location': x} for x  in subs]})
r.raise_for_status()
subs_location = r.json()['locations']

In [9]:
good_locations = []

for x in subs_location:
    if len(x) == 1:
        print('error here, deleting for now')
        print(x)
        tab_incidents.delete(subdivisions=x['location'])
    else:
        # rename
        x['subdivisions'] = x.pop('location')
        good_locations.append(x)

error here, deleting for now
{'location': 'Bon Courage fassungslos über Naziübergriffe auf dem Bornaer Stadtfest, LK Leipzig, Sachsen, Deutschland'}


In [10]:
tab_loc = db['locations']
tab_loc.insert_many(good_locations)

In [11]:
tab_loc.create_index(['subdivisions'])

In [56]:
# # Not really sure whether a seperate table for location is needed. It was introduced because in some cases, multiple locations are associated with incident.
# for x in tab_data.all():
#     row_loc = tab_loc.find_one(identifier=x['identifier'])
#     merged = {**row_loc, **x}
#     tab_data.update(merged, ['id'])